In [2]:
from imblearn.over_sampling import RandomOverSampler
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = ""
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import MaxPooling2D
from keras.layers import Conv2D
from keras.models import load_model
from keras.models import Sequential
from keras.utils import to_categorical
from skimage.transform import resize
from sklearn.model_selection import train_test_split
import csv
import h5py
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [3]:
# set global variables and hyper-parameters
DATA_LOCATION = '../data/'
TRAIN_IMAGES_LOCATION = '../data/train_images/'
IMAGE_SIZE = 64
N_CLASSES = 121
BATCH_SIZE = 32
N_EPOCHS = 20

# load train images filenames with class labels
filenames = [i for i in os.listdir('../data/train_images') if i.endswith('.jpg')]
with open(DATA_LOCATION + 'train_onelabel.csv', mode='r') as infile:
    reader = csv.reader(infile)
    file_to_class = {rows[0]:rows[1] for rows in reader}
    
infile.close()

# 1 if False, 4 if True
add_rotations = 1

X = np.empty([len(filenames)*add_rotations,IMAGE_SIZE,IMAGE_SIZE,1])
Y_tmp = np.empty([len(filenames)*add_rotations])
Y = np.empty([191180,N_CLASSES])
print('Shapes:\nX:{}\nY:{}'.format(X.shape, Y.shape))

Shapes:
X:(24204, 64, 64, 1)
Y:(191180, 121)


In [4]:
def get_padding(i):
    """
    Helper function for getting right padding sizes
    input:
        - i: positive integer gotten from substracting height and width of an image
    output:
        - Tuple representing the correct padding
    """
    if i%2 == 0:
        return (int(i/2),int(i/2))
    else:
        return (int(i/2-.5), int(i/2+.5))

def pad_image(img):
    """
    Add padding to image to make it square
    input:
        - img: numpy array (2D) representing image
    output:
        - padded array of shape (N,N)
    """
    H, W = img.shape
    if H == W:
        return img
    elif H > W:
        return np.pad(img, ((0,0), get_padding(H-W)), 'constant')
    else:
        return np.pad(img, (get_padding(W-H), (0,0)), 'constant')

def resize_image(img):
    """
    Resize image to new square shape
    input:
        - img: numpy array (2D) representing image
        - size: final shape of image in pixels (integer)
    """
    return resize(img, (IMAGE_SIZE,IMAGE_SIZE), mode='reflect')

For image in filenames:
- load file
- from [0,255] to [0.0 to 1.0]
- square and resize image
- rotate [0,90,180,270]
- add 4 images to X
- look up label in .csv
- add 4 labels to Y

`numpy.rot90(numpy.ndarray, nr_of_90_degrees_rotations)` might be a better option.

In [5]:
total = len(filenames)
print(total)
c=0
# for i in range(total):
#     if c%1000 == 0:
#         print( c)
#     # read and transform image to usable format
#     img = mpimg.imread(TRAIN_IMAGES_LOCATION + filenames[i])
#     img = np.absolute(np.divide(img.astype(float), 255) - 1.0)
#     img = resize_image(pad_image(img))
#     # create a grayscale channel 
#     img = img.reshape(64,64,1)
    
#     # save 4 times, for rotation of [0,90,180,270]
#     for j in range(add_rotations):
#         X[i+j*total] = np.rot90(img,j)
#         # Y[i+j*total][int(file_to_class[filenames[i]])] = 1.0
#         Y_tmp[i+j*total] = int(file_to_class[filenames[i]])
#     c+=1

24204


In [6]:
X = np.load("X.npy")
Y = np.load("Y.npy")

In [ ]:
f = plt.figure(figsize=(16,3))
sub1 = plt.subplot(1,4,1)
plt.imshow(X[250][:,:,0], cmap='binary')
if(add_rotations > 1):
    sub2 = plt.subplot(1,4,2)
    plt.imshow(X[250+len(filenames)*1][:,:,0], cmap='binary')
    sub3 = plt.subplot(1,4,3)
    plt.imshow(X[250+len(filenames)*2][:,:,0], cmap='binary')
    sub4 = plt.subplot(1,4,4)
    plt.imshow(X[250+len(filenames)*3][:,:,0], cmap='binary')

Over sample minority classes to be as big as the majority class. <br>
Please do note that the validation split accuracy can not be seen as a surrogate for the test set.

In [ ]:
X = X.reshape(total,IMAGE_SIZE*IMAGE_SIZE)
print(X.shape)

sm = RandomOverSampler()
X, Y_tmp = sm.fit_sample(X, Y_tmp)
print('Shapes:\nX:{}\nY:{}'.format(X.shape, Y_tmp.shape))

X = X.reshape(len(X),IMAGE_SIZE,IMAGE_SIZE,1)
for i in range(len(Y_tmp)):
    Y[i][int(Y_tmp[i])] = 1.0
del(Y_tmp)
print('Shapes:\nX:{}\nY:{}'.format(X.shape, Y.shape))

In [ ]:
for i in range(total,X.shape[0]):
    # rotate RandomOverSampler images by one of 90/180/270 degrees
    X[i] = np.rot90(X[i],(1+(i%4)))

In [7]:
model = Sequential()

# if validation set, input_shape=X_train[0], else X[0].shape
model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu', input_shape=X[0].shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(16, kernel_size=(3, 3), padding='same', activation='relu'))

model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(N_CLASSES, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 64)        640       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        18464     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 16)        4624      
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 16)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4096)              0         
__________

In [8]:
history = model.fit(
    X, # X_train 
    Y, # y_train
#    validation_data=(X_test,y_test),
    epochs=10, # N_EPOCHS
    batch_size=BATCH_SIZE,
    verbose=1)

Epoch 1/10
  2592/191180 [..............................] - ETA: 1:00:30 - loss: 4.7387 - acc: 0.0135    

KeyboardInterrupt: 

In [ ]:
model.save('../data/output/models/model9.h5')

In [ ]:
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()